In [ ]:
# default_exp utils

# Utils

> Collection of useful functions.

In [ ]:
#hide
from nbdev.showdoc import *

from typing import Iterable
from plum import dispatch

function = type(lambda: ())

In [ ]:
@dispatch
def simplify(x): return x

@dispatch
def simplify(fn: function): 
    try:
        return fn()
    except TypeError:
        return fn

@dispatch
def simplify(i: Iterable): return next(i.__iter__()) if len(i) == 1 else i
